### Remove y characters

In [1]:
with open('DATALOG.txt', 'rb') as file:
    content_bytes = file.read()

content = content_bytes.decode('utf-8', errors='ignore')

cleaned_content = content.replace('�', '')

with open('DATALOG.txt', 'wb') as file:
    file.write(cleaned_content.encode('utf-8'))

print("Undesired characters removed from DATALOG.txt")


Undesired characters removed from DATALOG.txt



### Find the Mean Value

In [2]:
import pandas as pd
df = pd.read_csv('datalog.csv')

mean_value = df['Temp'].mean()

print("Mean value of the column:", mean_value)

Mean value of the column: 55.181777777777775


### Sorting

In [2]:
import pandas as pd

csv_file = '../sensor_data_hours_updated.csv'
sorted_csv_file = '../sensor_data_hours_updated.csv'

df = pd.read_csv(csv_file)

sorted_df = df.sort_values(by=['Timestamp'])

sorted_df.to_csv(sorted_csv_file, index=False)

print(f'CSV file sorted and saved to {sorted_csv_file}')



CSV file sorted and saved to ../sensor_data_hours_updated.csv


### Turning data from Data Logger to Data Visualizer

#### Translating sensor number

In [30]:
import pandas as pd

# Dictionary of sensor numbers and their corresponding coordinates
sensor_coordinates = {
    'S_1': (0, 0, 3),
    'S_2': (1, 0, 1),
    'S_3': (2, 0, 3),
    'S_4': (3, 0, 1),
    'S_5': (0, 1, 1),
    'S_6': (1, 1, 3),
    'S_7': (2, 1, 1),
    'S_8': (3, 1, 3),
    'S_9': (0, 2, 3),
    'S_10': (1, 2, 1),
    'S_11': (2, 2, 3),
    'S_12': (3, 2, 4),
    'S_13': (0, 3, 1),
    'S_14': (1, 3, 3),
    'S_15': (2, 3, 1),
    'S_16': (3, 3, 3),
    'S_17': (0, 4, 3),
    'S_18': (1, 4, 1),
    'S_19': (2, 4, 3),
    'S_20': (3, 4, 1)
}

def calculate_true_sensor(sensor_number):
    return int(sensor_number.split('_')[1])

df = pd.read_csv('datalog.csv', delimiter=',')

df['X'] = pd.Series(dtype=int)
df['Y'] = pd.Series(dtype=int)
df['Z'] = pd.Series(dtype=int)

for sensor_number, (X, Y, Z) in sensor_coordinates.items():
    true_sensor_number = calculate_true_sensor(sensor_number)
    df.loc[df['Sensor_Number'] == true_sensor_number, ['X', 'Y', 'Z']] = [X, Y, Z]

df.drop(columns=['Sensor_Number'], inplace=True)

df.to_csv('new_datalog_with_coordinates.csv', index=False)



#### Add coordinate to sensor number

In [46]:
import csv

# Dictionary containing coordinates for each sensor number
sensor_coordinates = {
    '1': (0, 0, 3),
    '2': (1, 0, 1),
    '3': (2, 0, 3),
    '4': (3, 0, 1),
    '5': (0, 1, 1),
    '6': (1, 1, 3),
    '7': (2, 1, 1),
    '8': (3, 1, 3),
    '9': (0, 2, 3),
    '10': (1, 2, 1),
    '11': (2, 2, 3),
    '12': (3, 2, 1),
    '13': (0, 3, 1),
    '14': (1, 3, 3),
    '15': (2, 3, 1),
    '16': (3, 3, 3),
    '17': (0, 4, 3),
    '18': (1, 4, 1),
    '19': (2, 4, 3),
    '20': (3, 4, 1)
}

# Function to add coordinates to each row of the CSV file
def add_coordinates_to_csv(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # Write header
        writer.writerow(['Sensor_Number', 'Temp', 'Date', 'X', 'Y', 'Z'])

        # Iterate through each row
        for row in reader:
            sensor_number = row[0]
            if sensor_number.isdigit():  # Check if the row is not the header
                coordinates = sensor_coordinates.get(sensor_number)
                if coordinates:
                    writer.writerow(row + list(coordinates))

# Usage
add_coordinates_to_csv('slave_removed.csv', 'output_with_coordinates.csv')


#### Split timestamp into separate columns

In [47]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("output_with_coordinates.csv")

# Remove leading and trailing whitespace from the 'Date' column
df['Date'] = df['Date'].str.strip()

# Parse the original date string into a datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y %H:%M:%S')

# Extract year, month, day, hour, minute, and second into separate columns
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Hour'] = df['Date'].dt.hour
df['Minute'] = df['Date'].dt.minute
df['Second'] = df['Date'].dt.second

# Rearrange the columns as needed
df = df[['Sensor_Number','X','Y','Z','Year','Month','Day','Hour','Minute','Second','Temp','Date']]
df = df.rename(columns={'Sensor_Number':'Sensor_number','Date':'Timestamp','Temp':'Temperature'})

# Save the modified DataFrame to a new CSV file
df.to_csv("modified_file.csv", index=False)


### Clean minutes and seconds

In [45]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("modified_file.csv")

# Pad minute and second columns with leading zeros
df['Minute'] = df['Minute'].apply(lambda x: '{:02d}'.format(x))
df['Second'] = df['Second'].apply(lambda x: '{:02d}'.format(x))

# Pad timestamp column with leading zeros for minute and second
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Timestamp'] = df['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Write the updated DataFrame back to a CSV file
df.to_csv("cleared_ms.csv", index=False)
